In [2]:
import deap
import cma
import gym, gym_fastsim
from deap import *
import numpy as np
from scipy.spatial import KDTree
from copy import deepcopy

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

import datetime
from quadtree import *
from deap import algorithms
from deap import base
from deap import benchmarks
from deap import creator
from deap import tools

import array
import random
import operator
import math
import os.path


from scoop import futures

#from novelty_search_vanila import *
import os

USE_CUDA = False
def to_numpy(var):
    return var.cpu().data.numpy() if USE_CUDA else var.data.numpy()

In [3]:
!pip install kdtree

In [4]:
from copy import deepcopy

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np


if torch.cuda.is_available():
    FloatTensor = torch.cuda.FloatTensor
else:
    FloatTensor = torch.FloatTensor


class TNN(nn.Module):

    def __init__(self, **kwargs):
        super(TNN, self).__init__()
        self.state_dim = kwargs["nb_input"]
        self.action_dim = kwargs["nb_output"]
        self.max_action = kwargs["max_action"]

    def set_parameters(self, params):
        """
        Set the params of the network to the given parameters
        """
        cpt = 0
        for param in self.parameters():
            tmp = np.product(param.size())

            if torch.cuda.is_available():
                param.data.copy_(torch.from_numpy(
                    params[cpt:cpt + tmp]).view(param.size()).cuda())
            else:
                param.data.copy_(torch.from_numpy(
                    params[cpt:cpt + tmp]).view(param.size()))
            cpt += tmp

    def get_parameters(self):
        """
        Returns parameters of the actor
        """
        return deepcopy(np.hstack([to_numpy(v).flatten() for v in
                                   self.parameters()]))

    def get_grads(self):
        """
        Returns the current gradient
        """
        return deepcopy(np.hstack([to_numpy(v.grad).flatten() for v in self.parameters()]))

    def get_size(self):
        """
        Returns the number of parameters of the network
        """
        return self.get_parameters().shape[0]

    def load_model(self, filename, net_name):
        """
        Loads the model
        """
        if filename is None:
            return
        super(RLNN, self).load_model(filename=filename)

    def save_model(self, output, net_name):
        """
        Saves the model
        """
        super(RLNN, self).save_model(filename=output)


class Regression(TNN):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        self.theta = nn.Parameter(torch.randn(kwargs["nb_input"], requires_grad=True, dtype=torch.float).view((kwargs["nb_input"],1)))
        self.biais = nn.Parameter(torch.randn(kwargs["nb_output"]))
               
    def predict(self, x):
        x = torch.tensor(x)
        pred = torch.matmul(x,self.theta) + self.biais
        pred = pred.detach().numpy()
        return pred



In [5]:
def eval_nn(genotype,nn_class, resdir, render=False, dump=False, name="", **kwargs):
    """ Evaluation of a neural network. Returns the fitness, the behavior descriptor and a log of what happened
        Consider using dump=True to generate log files. These files are put in the resdir directory.
    """
    genotype = np.array(genotype)
    env = gym.make(kwargs['gym_name'], **kwargs['env_params'])
    nbstep=kwargs["episode_nb_step"]
    nn=nn_class(**kwargs)
    nn.set_parameters(genotype)
    observation = env.reset()
    observation, reward, done, info = env.step([0]*kwargs["nb_output"])

    #print("First observation: "+str(observation)+" first pos: "+str(env.get_robot_pos()))
    if (dump):
        f={}
        for k in info.keys():
            fn=resdir+"/traj_"+k+"_"+name+".log"
            if (os.path.exists(fn)):
                cpt=1
                fn=resdir+"/traj_"+k+"_"+name+"_%d.log"%(cpt)
                while (os.path.exists(fn)):
                    cpt+=1
                    fn=resdir+"/traj_"+k+"_"+name+"_%d.log"%(cpt)
            f[k]=open(fn,"w")

    action_scale_factor = env.action_space.high
    episode_reward=0
    episode_bd=None
    episode_log={}
    for t in range(nbstep):
        if render:
            env.render()
        action=nn.predict(observation)
        action=action_scale_factor*np.array(action)
        #print("Observation: "+str(observation)+" Action: "+str(action))
        observation, reward, done, info = env.step(action) 
        if (kwargs["episode_reward_kind"] == "cumul"):
            episode_reward+=reward

        for k in kwargs["episode_log"].keys():
            if (kwargs["episode_log"][k] == "cumul"):
                if (k not in episode_log.keys()):
                    episode_log[k] = info[k]
                else:
                    episode_log[k] += info[k]
        if(dump):
            for k in f.keys():
                if (isinstance(info[k], list) or isinstance(info[k], tuple)):
                    data=" ".join(map(str,info[k]))
                else:
                    data=str(info[k])
                f[k].write(data+"\n")
        if(done):
            break
    if (dump):
        for k in f.keys():
            f[k].close()

    if (kwargs["episode_reward_kind"] == "final"):
        episode_reward=reward
        
    if (kwargs["episode_bd_kind"] == "final"):
        episode_bd=info[kwargs["episode_bd"]][slice(*kwargs["episode_bd_slice"])]
        
    for k in kwargs["episode_log"].keys():
        if (kwargs["episode_log"][k] == "final"):
            episode_log[k] = info[k]
    
    if(episode_log["exit_reached"]==1.0):
            print("Target REACHED ! ")
    #print("End of eval, t=%d, total_dist=%f"%(t,total_dist))
    return episode_reward, episode_bd, episode_log

array([ 0.19354811, -0.4204001 ,  0.6847287 ,  0.33281985,  0.26104757],
      dtype=float32)

In [6]:
kw={
    'gym_name': 'FastsimSimpleNavigation-v0',
    'env_params': {"still_limit": 10, 'reward_kind': "continuous"},

    'nb_input': 5, # number of NN inputs
    'nb_output': 2, # number of NN outputs
    'nb_layers': 2, # number of layers
    'nb_neurons_per_layer': 10, # number of neurons per layer
    'max_action': 2,
    'episode_nb_step': 1000, 
    'episode_reward_kind': 'final', 
    'episode_bd': 'robot_pos', 
    'episode_bd_slice': (0,2,None), 
    'episode_bd_kind': 'final', 
    'episode_log': {'collision': 'cumul', 
                    'dist_obj': 'final', 
                    'exit_reached': 'final', 
                    'robot_pos': 'final'},
    'dim_grid': [100, 100],
    'grid_min_v': [0,0],
    'grid_max_v': [600,600],
    'goal': [60,60],
    'watch_max': 'dist_obj',
    'min_value': -30, # min genotype value
    'max_value': 30, # max genotype value
    'min_strategy': 0.5, # min value for the mutation
    'max_strategy': 3, # max value for the mutation
    'nb_gen': 25, # number of generations
    'mu': 100, # population size
    'lambda': 200, # number of individuals generated
    'nov_k': 15, # k parameter of novelty search
    'nov_add_strategy': "random", # archive addition strategy (either 'random' or 'novel')
    'nov_lambda': 6, # number of individuals added to the archive
    'selection' : "blablabla"
}

In [7]:
t = Regression(**kw)
gen = t.get_parameters()
eval_nn(gen,Regression, resdir, render=False, dump=False, name="", **kw)

NameError: name 'resdir' is not defined

In [187]:
# Individual generator
def generateES(icls, scls, size, imin, imax, smin, smax):
    ind = icls(random.uniform(imin, imax) for _ in range(size))
    ind.strategy = scls(random.uniform(smin, smax) for _ in range(size))
    return ind

def checkStrategy(minstrategy):
    def decorator(func):
        def wrappper(*args, **kargs):
            children = func(*args, **kargs)
            for child in children:
                for i, s in enumerate(child.strategy):
                    if s < minstrategy:
                        child.strategy[i] = minstrategy
            return children
        return wrappper
    return decorator



def launch_ea(nn_class,selector_class, mu=100, lambda_=200, cxpb=0.3, mutpb=0.7, ngen=100, verbose=False, resdir="res",weights=(1.0,1.0), **kwargs):

    random.seed()
    selector = selector_class(**kwargs)


    creator.create("MyFitness", base.Fitness, weights=weights)

    creator.create("Individual", array.array, typecode="d", fitness=creator.MyFitness, strategy=None)
    creator.create("Strategy", array.array, typecode="d")
    nn=nn_class(**kwargs)
    center=nn.get_parameters()
    toolbox = base.Toolbox()
    toolbox.register("individual", generateES, creator.Individual, creator.Strategy, nn.get_size(), 
                     kwargs["min_value"], 
                     kwargs["max_value"], 
                     kwargs["min_strategy"], 
                     kwargs["max_strategy"])

    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("mate", tools.cxESBlend, alpha=0.1)
    toolbox.register("mutate", tools.mutESLogNormal, c=1.0, indpb=0.03)
    toolbox.register("select", selector.select)
    
    toolbox.register("map",futures.map)
    toolbox.decorate("mate", checkStrategy(kwargs["min_strategy"]))
    toolbox.decorate("mutate", checkStrategy(kwargs["min_strategy"]))


    toolbox.register("evaluate", eval_nn, resdir=resdir, nn_class=nn_class, **kwargs)


    population = toolbox.population(n=mu)
    paretofront = tools.ParetoFront()
    
    fbd=open(resdir+"/bd.log","w")
    finfo=open(resdir+"/info.log","w")
    ffit=open(resdir+"/fit.log","w")

    nb_eval=0

    ##
    ### Initial random generation: beginning
    ##

    # Evaluate the individuals with an invalid (i.e. not yet evaluated) fitness
    invalid_ind = [ind for ind in population if not ind.fitness.valid]
    fitnesses_bds = toolbox.map(toolbox.evaluate, invalid_ind)
    nb_eval+=len(invalid_ind)
    finfo.write("## Generation 0 \n")
    finfo.flush()
    ffit.write("## Generation 0 \n")
    ffit.flush()
    for ind, (fit, bd, log) in zip(invalid_ind, fitnesses_bds):
        #print("Fit: "+str(fit)) 
        #print("BD: "+str(bd))
        ind.fit = fit
        ind.log = log
        ind.bd = bd
        fbd.write(" ".join(map(str,bd))+"\n")
        fbd.flush()
        finfo.write(str(log)+"\n")
        finfo.flush()
        ffit.write(str(fit)+"\n")
        ffit.flush()

    if paretofront is not None:
        paretofront.update(population)
    
    #Améliorer ça 
    #archive=updateNovelty(population,population,None,
    #                      registered_envs[env_name]['nov_k'],
    #                      registered_envs[env_name]['nov_add_strategy'],
    #                      registered_envs[env_name]['nov_lambda'])
    selector.update_with_offspring(population)
    selector.compute_objectifs(population)
    #for ind in population:
    #    if (kwargs['selection']=="FIT+NS"):
    #        ind.fitness.values=(ind.fit,ind.novelty)
    #    elif (kwargs['selection']=="FIT"):
    #        ind.fitness.values=(ind.fit,)
    #    elif (kwargs['selection']=="NS"):
    #        ind.fitness.values=(ind.novelty,)

    #print("Fit=%f Nov=%f"%(ind.fit, ind.novelty))

    indexmax, valuemax = max(enumerate([i.log[kwargs['watch_max']] for i in population]), key=operator.itemgetter(1))


    # Begin the generational process
    for gen in range(1, ngen + 1):
        finfo.write("## Generation %d \n"%(gen))
        finfo.flush()
        ffit.write("## Generation %d \n"%(gen))
        ffit.flush()
        if (gen%10==0):
            print("+",end="", flush=True)
        else:
            print(".",end="", flush=True)

        # Vary the population
        offspring = algorithms.varOr(population, toolbox, lambda_, cxpb, mutpb)

        # Evaluate the individuals with an invalid (i.e. not yet evaluated) fitness
        invalid_ind = [ind for ind in offspring]
        fitnesses_bds = toolbox.map(toolbox.evaluate, invalid_ind)
        nb_eval+=len(invalid_ind)

        for ind, (fit, bd, log) in zip(invalid_ind, fitnesses_bds):
            #print("Fit: "+str(fit)+" BD: "+str(bd)) 
            ind.fit = fit
            ind.bd = bd
            ind.log=log
            fbd.write(" ".join(map(str,bd))+"\n")
            fbd.flush()
            finfo.write(str(log)+"\n")
            finfo.flush()
            ffit.write(str(fit)+"\n")
            ffit.flush()
            #grid_management.add_to_grid(grid,ind, ind.fit, 
            #                            dim=kwargs['dim_grid'], 
            #                            min_v=kwargs['grid_min_v'], 
            #                            max_v=kwargs['grid_max_v'])
        pq=population+offspring

        #archive=updateNovelty(pq,offspring,archive,
        #                      registered_envs[env_name]['nov_k'],
        #                      registered_envs[env_name]['nov_add_strategy'],
        #                      registered_envs[env_name]['nov_lambda'])
        
        selector.update_with_offspring(pq)
        selector.compute_objectifs(pq)

        #for ind in pq:
        #    if (kwargs['selection']=="FIT+NS"):
        #        ind.fitness.values=(ind.fit,ind.novelty)
        #    elif (kwargs['selection']=="FIT"):
        #        ind.fitness.values=(ind.fit,)
        #    elif (kwargs['selection']=="NS"):
        #        ind.fitness.values=(ind.novelty,)

        #print("Fitness values: "+str(ind.fitness.values))
        

        # Select the next generation population
        population[:] = toolbox.select(pq, mu)

        indexmax, newvaluemax = max(enumerate([i.log[kwargs['watch_max']] for i in pq]), key=operator.itemgetter(1))
        if (newvaluemax>valuemax):
            valuemax=newvaluemax
            print("Gen "+str(gen)+", new max ! max fit="+str(valuemax)+" index="+str(indexmax)+" BD="+str(pq[indexmax].bd))
            nnfit, nnbd, log = eval_nn(pq[indexmax],nn_class,resdir = resdir,render=True,dump=True,name="gen%04d"%(gen),**kwargs)
    fbd.close()
    finfo.close()
    ffit.close()

    
    #grid_management.stat_grid(grid, resdir, nb_eval, dim=kwargs['dim_grid'])
    #grid_management.dump_grid(grid, resdir, dim=kwargs['dim_grid'])

    return population, selector

In [188]:
from scipy.spatial import KDTree
import random
import numpy as np
from operator import itemgetter

class NovArchive:
    """Archive used to compute novelty scores."""
    def __init__(self, k=15):
        self.all_bd=lbd
        self.kdtree=KDTree(self.all_bd)
        self.k=k
        #print("Archive constructor. size = %d"%(len(self.all_bd)))

    def update(self,new_bd):
        oldsize=len(self.all_bd)
        self.all_bd=self.all_bd + new_bd
        self.kdtree=KDTree(self.all_bd)
        #print("Archive updated, old size = %d, new size = %d"%(oldsize,len(self.all_bd)))

    def get_nov(self,bd, population=[]):
        dpop=[]
        for ind in population:
            dpop.append(np.linalg.norm(np.array(bd)-np.array(ind.bd)))
        darch,ind=self.kdtree.query(np.array(bd),self.k)
        d=dpop+list(darch)
        d.sort()
        if (d[0]!=0):
            print("WARNING in novelty search")
        return sum(d[:self.k+1])/self.k # as the indiv is in the population, the first value is necessarily a 0.

    def size(self):
        return len(self.all_bd)
    
def updateNovelty(population, offspring, archive, k=15, add_strategy="random", _lambda=6, verbose=False):
   """Update the novelty criterion (including archive update) 

   Implementation of novelty search following (Gomes, J., Mariano, P., & Christensen, A. L. (2015, July). Devising effective novelty search algorithms: A comprehensive empirical study. In Proceedings of GECCO 2015 (pp. 943-950). ACM.).
   :param population: is the set of indiv for which novelty needs to be computed
   :param offspring: is the set of new individuals that need to be taken into account to update the archive (may be the same as population, but it may also be different as population may contain the set of parents)
   :param k: is the number of nearest neighbors taken into account
   :param add_strategy: is either "random" (a random set of indiv is added to the archive) or "novel" (only the most novel individuals are added to the archive).
   :param _lambda: is the number of individuals added to the archive for each generation
   The default values correspond to the one giving the better results in the above mentionned paper.

   The function returns the new archive
   """
   
   # Novelty scores updates
   if (archive) and (archive.size()>=k):
       if (verbose):
           print("Update Novelty. Archive size=%d"%(archive.size())) 
       for ind in population:
           ind.novelty=archive.get_nov(ind.bd, population)
           #print("Novelty: "+str(ind.novelty))
   else:
       if (verbose):
           print("Update Novelty. Initial step...") 
       for ind in population:
           ind.novelty=0.

   if (verbose):
       print("Fitness (novelty): ",end="") 
       for ind in population:
           print("%.2f, "%(ind.novelty),end="")
       print("")
   if (len(offspring)<_lambda):
       print("ERROR: updateNovelty, lambda(%d)<offspring size (%d)"%(_lambda, len(offspring)))
       return None

   lbd=[]
   # Update of the archive
   if(add_strategy=="random"):
       # random individuals are added
       l=list(range(len(offspring)))
       random.shuffle(l)
       if (verbose):
           print("Random archive update. Adding offspring: "+str(l[:_lambda])) 
       lbd=[offspring[l[i]].bd for i in range(_lambda)]
   elif(add_strategy=="novel"):
       # the most novel individuals are added
       soff=sorted(offspring,lambda x:x.novelty)
       ilast=len(offspring)-_lambda
       lbd=[soff[i].bd for i in range(ilast,len(soff))]
       if (verbose):
           print("Novel archive update. Adding offspring: ")
           for offs in soff[iLast:len(soff)]:
               print("    nov="+str(offs.novelty)+" fit="+str(offs.fitness.values)+" bd="+str(offs.bd))
   else:
       print("ERROR: updateNovelty: unknown add strategy(%s), valid alternatives are \"random\" and \"novel\""%(add_strategy))
       return None
       
   if(archive==None):
       archive=NovArchive(lbd,k)
   else:
       archive.update(lbd)

   return archive

In [189]:
from grid_management import Grid


class Novelty_Archive():
    def __init__(self, k=15, lambda_=10, **kwargs):
        self.all_bd={}
        self.kdtree=None
        self.k=k
        self.lambda_ = lambda_
        #print("Archive constructor. size = %d"%(len(self.all_bd)))

    def update(self,ind):
        oldsize=len(self.all_bd)
        if(self.kdtree == None):
            self.all_bd[tuple(ind.bd)] = self.all_bd.get(tuple(ind.bd),[]) + [ind]
            self.kdtree=KDTree([ind.bd])
        else:
            self.all_bd[tuple(ind.bd)] = self.all_bd.get(tuple(ind.bd),[]) + [ind]
            self.kdtree=KDTree(list(self.all_bd.keys()))
        #print("Archive updated, old size = %d, new size = %d"%(oldsize,len(self.all_bd)))
    
    
    def select_from_offsprint(self, offspring):
        soff=sorted(offspring,key=lambda x:x.novelty)
        ilast=len(offspring)-self.lambda_
        lbd=[soff[i] for i in range(ilast,len(soff))]
        return lbd
    
    def update_offspring(self, offspring):
        if(self.kdtree == None):
            self.update(offspring[0])
        self.apply_novelty_estimation(offspring)
        lbd = self.select_from_offsprint(offspring)
        for i in lbd:
            self.update(i)

    def apply_novelty_estimation(self, population):
        for ind in population:
            ind.novelty = self.get_nov(ind.bd, population)
        

    def get_nov(self,bd, population=[]):
        dpop=[]
        for ind in population:
            dpop.append(np.linalg.norm(np.array(bd)-np.array(ind.bd)))
        darch,ind=self.kdtree.query(np.array(bd),self.k)
        d=dpop+list(darch)
        d.sort()
        if (d[0]!=0):
            print("WARNING in novelty search: the smallest distance should be 0 (distance to itself).")
        return sum(d[:self.k+1])/self.k # as the indiv is in the population, the first value is necessarily a 0.

    def size(self):
        return len(self.all_bd)



class Novelty_Archive_random(Novelty_Archive):

    def __init__(self, k=15, lambda_ = 10):
        super().__init__(k=k, lambda_=lambda_)
    
    def select_from_offsprint(self, offspring, verbose=False):
        l=list(range(len(offspring)))
        random.shuffle(l)
        if (verbose):
            print("Random archive update. Adding offspring: "+str(l[:lambda_])) 
        lbd=[offspring[l[i]] for i in range(self.lambda_)]
        return lbd

In [190]:
class Selector():
    def __init__(self, archive_class, **kwargs):
        self.archive = archive_class(k=kwargs["nov_k"], lambda_ = kwargs["nov_lambda"])
        self.grid = Grid(kwargs["grid_min_v"],kwargs["grid_max_v"],kwargs["dim_grid"])

    def update_with_offspring(self, offspring):
        for ind in offspring:
            self.grid.add(ind)
        self.archive.update_offspring(offspring)
        pass

    def compute_objectifs(self, population):
        pass

    def select(self, pq, mu):
        return tools.selNSGA2(pq,mu)


class Selector_FITNS(Selector):

    def __init__(self, **kwargs):
        super().__init__(Novelty_Archive_random, **kwargs)


    def compute_objectifs(self, population):
        self.archive.apply_novelty_estimation(population)
        for i in population:
            i.fitness.values = (i.fit, i.novelty)


class Selector_NS(Selector):

    def __init__(self, **kwargs):
        super().__init__(Novelty_Archive_random, **kwargs)


    def compute_objectifs(self, population):
        self.archive.apply_novelty_estimation(population)
        for i in population:
            i.fitness.values = (i.novelty, )

In [191]:
S = Selector_FITNS(**kw)

In [192]:
print(S)

In [193]:
class Regression(TNN):
    def __init__(self,**kwargs):
        super().__init__(**kwargs)
        self.relu = nn.ReLU()
        self.sigm = nn.Sigmoid()
        self.max_action = kwargs["max_action"]
        self.l1 = nn.Linear(kwargs["nb_input"],10)
        self.l2 = nn.Linear(10,10)
        self.l3 = nn.Linear(10,10)
        self.out = nn.Linear(10,kwargs["nb_output"])

               
    def predict(self, x):
        x = torch.tensor(x)
        x = self.sigm(self.l1(x))
        x = self.sigm(self.l2(x))
        x = self.sigm(self.l3(x))

        x = self.out(x)
        x = torch.tanh(x)
        x = x.detach().numpy()
        return x*self.max_action



In [194]:
!mkdir res

mkdir: cannot create directory ‘res’: File exists


In [195]:
import os
import datetime
resdir="res/RUN_FITNS_"+"_"+datetime.datetime.now().strftime("%Y_%m_%d_%H:%M:%S")
os.mkdir(resdir)
pop,select = launch_ea(Regression,Selector_FITNS, ngen=300, resdir=resdir, **kw)

.

KeyboardInterrupt: 

In [17]:
import os
import datetime
resdir="res/RUN_FITNS_"+"_"+datetime.datetime.now().strftime("%Y_%m_%d_%H:%M:%S")
os.mkdir(resdir)
pop,select = launch_ea(Regression,Selector_FITNS, ngen=300, resdir=resdir, **kw)

.Gen 1, new max ! max fit=-250.14172861357792 index=113 BD=[53.09649658203125, 310.04644775390625]
.Gen 2, new max ! max fit=-207.51175415181962 index=101 BD=[16.59391212463379, 262.9212646484375]
.Gen 3, new max ! max fit=-207.38359774511315 index=125 BD=[16.534854888916016, 262.7775573730469]
.Gen 4, new max ! max fit=-205.17020896930285 index=120 BD=[16.548538208007812, 260.51629638671875]
..Gen 6, new max ! max fit=-96.38557549870923 index=164 BD=[51.895973205566406, 156.04428100585938]
...+....Gen 14, new max ! max fit=-96.02497962049816 index=183 BD=[51.52172088623047, 155.64996337890625]
....Gen 18, new max ! max fit=-95.40316320509318 index=285 BD=[55.04642105102539, 155.27447509765625]
.+Gen 20, new max ! max fit=-95.33298263593458 index=196 BD=[47.668670654296875, 154.53208923339844]
..Gen 22, new max ! max fit=-94.93241224351243 index=180 BD=[51.24242401123047, 154.52760314941406]
.Gen 23, new max ! max fit=-94.67672745063368 index=215 BD=[54.570594787597656, 154.52091979980

In [49]:
import os
import datetime
resdir="res/RUN_NS"+"_"+datetime.datetime.now().strftime("%Y_%m_%d_%H:%M:%S")
os.mkdir(resdir)
pop,select = launch_ea(Regression,Selector_NS, ngen=100, resdir=resdir, weights=(1.0,0.0), **kw)

.Gen 1, new max ! max fit=-220.7126061498128 index=194 BD=[31.11292266845703, 278.8140563964844]
..Gen 3, new max ! max fit=-220.71225517814926 index=175 BD=[31.115604400634766, 278.8140563964844]
.Gen 4, new max ! max fit=-206.99255862400048 index=202 BD=[21.85400390625, 263.4472961425781]
..Gen 6, new max ! max fit=-125.87104797466405 index=246 BD=[131.31845092773438, 163.7169189453125]
...Gen 9, new max ! max fit=-112.65932698965432 index=290 BD=[19.44037628173828, 165.1049041748047]
+Gen 10, new max ! max fit=-108.96600436360309 index=127 BD=[88.98873901367188, 165.03924560546875]
.Gen 11, new max ! max fit=-96.78950960363167 index=208 BD=[43.41399383544922, 155.35781860351562]
.Gen 12, new max ! max fit=-95.57917456453559 index=171 BD=[48.11726379394531, 154.837646484375]
..Gen 14, new max ! max fit=-94.81628051793092 index=223 BD=[58.3642578125, 154.8021697998047]
.....+.........+.........+.....Gen 45, new max ! max fit=-94.68880600090958 index=239 BD=[54.802757263183594, 154.546

In [19]:
select.grid.get_stats("res",100)

Coverage: 24.50 % (2450 cells out of 10000) Max score: -95.08 Min score: -735.17 Total quality: -958508.30


In [20]:
resdir="res/RUN_NS_"+"_"+datetime.datetime.now().strftime("%Y_%m_%d_%H:%M:%S")
os.mkdir(resdir)
pop = launch_ea(Regression,Selector_NS, ngen=300, resdir=resdir, **kw)

.Gen 1, new max ! max fit=-276.6671122114719 index=235 BD=[72.56407928466797, 336.3816833496094]
.Gen 2, new max ! max fit=-218.68654349532363 index=161 BD=[18.010597229003906, 274.6175537109375]
....Gen 6, new max ! max fit=-207.04121550320565 index=197 BD=[17.485095977783203, 262.62908935546875]
...+.Gen 11, new max ! max fit=-111.8126407957602 index=121 BD=[16.787757873535156, 163.12501525878906]
.Gen 12, new max ! max fit=-104.07326364561399 index=291 BD=[17.039306640625, 154.7925262451172]
....Gen 16, new max ! max fit=-98.5600097098436 index=112 BD=[24.552684783935547, 151.9650115966797]
...+.........+.........+Gen 40, new max ! max fit=-94.93628482471519 index=228 BD=[51.507381439208984, 154.5556640625]
.........+.........+.........+.........+.........+...Gen 93, new max ! max fit=-59.064628506832356 index=265 BD=[17.757875442504883, 18.717639923095703]
...Gen 96, new max ! max fit=-42.877080461040116 index=281 BD=[57.77353286743164, 17.18076515197754]
...+.........+......Gen 11

In [21]:
resdir="res/RUN_NS_"+"_"+datetime.datetime.now().strftime("%Y_%m_%d_%H:%M:%S")
os.mkdir(resdir)
pop = launch_ea(Regression,Selector_NS, ngen=300, resdir=resdir, **kw)

.Gen 1, new max ! max fit=-222.57669206034228 index=111 BD=[18.18576431274414, 278.61370849609375]
...Gen 4, new max ! max fit=-206.45332204730232 index=188 BD=[18.99691390991211, 262.3406066894531]
...Gen 7, new max ! max fit=-206.41606942718263 index=248 BD=[19.084115982055664, 262.32025146484375]
.Gen 8, new max ! max fit=-204.11930236021436 index=107 BD=[17.165191650390625, 259.57421875]
.+.........+.Gen 21, new max ! max fit=-137.80960573988165 index=239 BD=[17.451385498046875, 191.0767059326172]
.....Gen 26, new max ! max fit=-107.13819383147123 index=131 BD=[18.444225311279297, 158.75074768066406]
.Gen 27, new max ! max fit=-102.26344563090912 index=297 BD=[19.7652587890625, 154.01583862304688]
..+..Gen 32, new max ! max fit=-96.85899261553716 index=122 BD=[45.702396392822266, 155.7979278564453]
.......+.........+.........+....Gen 64, new max ! max fit=-95.94995520486285 index=132 BD=[46.887847900390625, 155.0498046875]
.....+Gen 70, new max ! max fit=-95.25031786469725 index=26

In [22]:
resdir="res/RUN_NS_"+"_"+datetime.datetime.now().strftime("%Y_%m_%d_%H:%M:%S")
os.mkdir(resdir)
pop = launch_ea(Regression,Selector_NS, ngen=300, resdir=resdir, **kw)

.Gen 1, new max ! max fit=-217.92622229924223 index=122 BD=[16.784496307373047, 273.5983581542969]
.Gen 2, new max ! max fit=-209.87463369156055 index=158 BD=[17.085498809814453, 265.4402770996094]
.....Gen 7, new max ! max fit=-209.86403513670552 index=229 BD=[17.116596221923828, 265.4359436035156]
..Gen 9, new max ! max fit=-111.05183254204793 index=107 BD=[16.877246856689453, 162.33737182617188]
+.........Gen 19, new max ! max fit=-100.76675452373227 index=259 BD=[29.089645385742188, 155.90875244140625]
+........Gen 28, new max ! max fit=-100.5407303634031 index=262 BD=[20.625507354736328, 152.5099334716797]
.+....Gen 34, new max ! max fit=-99.0549380104272 index=241 BD=[23.82839584350586, 152.2144012451172]
.....+....Gen 44, new max ! max fit=-64.96244904185804 index=231 BD=[16.81157112121582, 108.527099609375]
.Gen 45, new max ! max fit=-62.16614682878998 index=250 BD=[18.871566772460938, 106.61632537841797]
.Gen 46, new max ! max fit=-58.601473243558274 index=256 BD=[18.384399414

In [59]:
import kdtree

class Representative(object):
    def __init__(self, x, y, data):
        self.ind_list = []

    def __len__(self):
        return len(self.coords)

    def __getitem__(self, i):
        if(len(self.ind_list ) == 0):
            print("Representative empty !!!")
        return self.coords[i]

    def __repr__(self):
        return 'Item({}, {}, {})'.format(self.coords[0], self.coords[1], self.data)

In [233]:
import numpy as np

class Shine_Archive(QuadTree):
    def __init__(self, width, height, alpha = 8, beta = 10):
        super().__init__([],width, height)
        rect = Rect(0, 0, width, height)
        self.alpha = alpha
        self.beta = beta
        self.size = 0
        self.root = Node(val=[], bounds=rect)

    
    def update_offspring(self,offspring):
        for i in offspring:
            I = Behaviour_Descriptor(i)
            self.add_node(I)
        

    def _split(self, root):
        if root.leaf:
            rects = root.bounds.split()
            for son, bounds_rect in zip(("nw", "ne", "sw", "se"), rects):
                setattr(root, son, Node(val=[], bounds=bounds_rect, level=root.level + 1))
            for val in root.val:
                for son in root.sons():
                    if val in root.bounds:
                        son.val.append(val)
                        if(len(son.val) > self.beta):
                            self._split(son)
                        break
            root.val.clear()
        else:
            print("Trying to split not a leaf")
    
    def remove_worst(self, node):
        if(len(node.val) == 0):
            return
        points = [(i[0], i[1]) for i in node.val]
        #dists = [ (i,np.sqrt((i[0] - (node.bounds[0] + node.bounds[2]))**2 +  (i[1] - (node.bounds[1] + node.bounds[3])) **2 )) for i in node.val]
        dists = [ (i,i.ind.fit) for i in node.val]
        #print("points : ", points)
        #print(dists)
        dists = (sorted(dists, key = lambda x:x[1]))
        pos = node.val.index(dists[0][0])
        node.val.remove(dists[0][0])
        #print("points after : ",node.val)
        pass


    def add_node(self, val):
        node = self.search(val)
        if(val not in node.val):
            node.val.append(val)
        if(len(node.val) >= self.beta):
            if(node.level <= self.alpha):
                self._split(node)
                self.size += 1
            else:
                self.remove_worst(node)
        
        if(len(node.val) > self.beta):
            print("size after : ",len(node.val), " , level : ",node.level)



In [242]:
class Selector_SHINE(Selector):
    def __init__(self, **kwargs):
        self.archive = Shine_Archive(600,600)

    def update_with_offspring(self, offspring):
        self.archive.update_offspring(offspring)
        pass

    def compute_objectifs(self, population):
        for i in population:
            n = self.archive.search(Behaviour_Descriptor(i))
            if(len(n.val) > 0):
                i.fitness.values = (self.archive.beta / (self.archive.beta*n.level + len(n.val) ),)
                #i.fitness.values = (-n.level,- len(n.val) )

            else:
                i.fitness.values = (-np.inf,)
        pass

    def select(self, pq, mu):
        return tools.selNSGA2(pq,mu)

In [243]:
class Behaviour_Descriptor(object):
    def __init__(self,ind):
        self.ind = ind

    def __len__(self):
        return len(self.ind)

    def __getitem__(self, i):
        return self.ind.bd[i]

    def __eq__(self, other):
        return self.ind[0] == other.ind[0] and self.ind[1] == other.ind[1]

        
    def __repr__(self):
        return 'BD({})'.format(self.ind.bd)

In [244]:
tree2 = Shine_Archive(1000,1000)
for i in pop:
    I = Behaviour_Descriptor(i)
    tree2.add_node(I)

In [245]:
S = Selector_SHINE(**kw)
S.update_with_offspring(pop)
S.compute_objectifs(pop)

In [252]:
resdir="res/RUN_SHINE_"+"_"+datetime.datetime.now().strftime("%Y_%m_%d_%H:%M:%S")
os.mkdir(resdir)
pop = launch_ea(Regression,Selector_SHINE, ngen=150, resdir=resdir,weights=(1.0,), **kw)

....Gen 4, new max ! max fit=-101.82983833153288 index=258 BD=[19.828388214111328, 153.57113647460938]
.....+.........+......Gen 26, new max ! max fit=-44.83779628753939 index=295 BD=[17.18109130859375, 73.30297088623047]
...+Gen 30, new max ! max fit=-41.18152517777981 index=275 BD=[19.49127197265625, 67.41356658935547]
.....Gen 35, new max ! max fit=-40.18488508465792 index=214 BD=[20.321840286254883, 66.36149597167969]
...Gen 38, new max ! max fit=-15.078122736885973 index=296 BD=[51.87108612060547, 72.69923400878906]
.+..Gen 42, new max ! max fit=-12.562944839883198 index=281 BD=[50.064414978027344, 67.68841552734375]
.......+.........+......

KeyboardInterrupt: 